## ETF Compete:  
## Feature engineering: 
## VR,  (Volume Ratio, 容量指標) 
## OBV (On Balance Volume 能量潮)

http://blog.cnyes.com/my/victor888victor/article163587

## n 天內漲跌關於 量 的相對指標
### Date: 18426_L

# 使用變數


$$$$
# vol_up = (n days 上漲的量總和) + 1/2 * (ndays 的量總和)
# vol_dn = (n days 下跌的量總和) + 1/2 * (ndays 的量總和)
# VR =   vol_up/vol_dn*1.0

# note: VR 可能大於100% ，一般介於0.7 ~ 3.5 之間
# 計算完 VR , 使用 min_max 做normalize

$$$$

# OBV_new = OBV_old $\pm$ Today's Volume (+, if today's price up) 

# Using min_max to do normalization

# OBV_MA12 =  OBV moving average 12
# OBV_diff =  OBV - OBV_MA12 (計算 OBV vs OBV 移動平均差異)

### 參數: 
### df: dataframe 
### n: n days window (normally, 24, using 12 here  )


###  return df 
### feature columns = [ ‘VR',  'OBV','OBV_MA12', 'OBV_diff']
### note： 0 ..11 Rows VR, OBV_MA12 value will be initial value

In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict
from time import time
from sklearn.preprocessing import MinMaxScaler

In [2]:
# read in csv, change column names
etf = pd.read_csv('/Users/LarryGuo/Desktop/nano_degree/Capstone_Talk/ETF_compete/twetf_utf8/tetfp.csv')
col_dtypes = OrderedDict(code=str, date=str, name=str, open=float, high=float, low=float, close=float, volume=int)

etf.columns=col_dtypes.keys()

etf.head()

,code,date,name,open,high,low,close,volume
0,50,20130102,元大台灣50,54.00,54.65,53.90,54.40,16487
1,50,20130103,元大台灣50,54.90,55.05,54.65,54.85,29020
2,50,20130104,元大台灣50,54.85,54.85,54.40,54.50,9837
3,50,20130107,元大台灣50,54.55,54.55,53.90,54.25,8910
4,50,20130108,元大台灣50,54.00,54.20,53.65,53.90,12507


In [3]:

etf= etf[:1286] # select 台灣50 
etf.tail()

,code,date,name,open,high,low,close,volume
1281,50,20180327,元大台灣50,83.00,83.40,82.95,83.40,3277
1282,50,20180328,元大台灣50,82.90,82.90,82.20,82.25,4161
1283,50,20180329,元大台灣50,82.25,82.35,81.80,82.10,4099
1284,50,20180330,元大台灣50,82.65,83.05,82.65,82.85,4994
1285,50,20180331,元大台灣50,82.85,83.05,82.75,82.95,878


In [4]:
etf.columns

Index(['code', 'date', 'name', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [5]:


def calc_vol_up_dn(n_df):
    
    up =   n_df[n_df['diff']>=0]['volume'].sum() + n_df['volume'].sum()*0.5
    down = n_df[n_df['diff']< 0]['volume'].sum() + n_df['volume'].sum()*0.5
    

    
    
    return up, down

def VR_OBV(df,n=12):
    start_time=time()
    
    

    df['vol_up'] = 1

    df['vol_dn'] = 1
    df['VR'] = 1.0
    df['prev_close'] = df['close'].shift(1)
    df['diff'] = df['close'] - df['prev_close']

    # calculate VR
    for j in range(n-1,len(df)):
    #for j in range(n-1,50):
        df['vol_up'].iloc[j] = calc_vol_up_dn(df[j-n+1:j+1])[0]
        df['vol_dn'].iloc[j] = calc_vol_up_dn(df[j-n+1:j+1])[1]

    
    df['VR'] = df['vol_up']/df['vol_dn']*1.0
    
    # check if ['vol_dn contains 0, if yes, assign 1 to the df['VR] 
    if 0 in df['vol_dn'] :  
        
        df['VR'].loc[df['vol_dn']==0] = 1

    # normalization to scale test_df['VR] in [0,1]
    min_max_VR = MinMaxScaler()
    df['VR'] = min_max_VR.fit_transform(df['VR'].values.reshape(-1,1))
    
    
    # calculate OBV
    
    df['diff_index'] = np.where (df['diff']>=0,1,-1)
    df['OBV'] = df['diff_index'] * df['volume']
    df['OBV'].iloc[0] = df['volume'].iloc[0]
    df['OBV'] = df['OBV'].cumsum()
    
    
    min_max_OBV = MinMaxScaler()
    df['OBV'] = min_max_OBV.fit_transform(df['OBV'].values.reshape(-1,1))
    df['OBV_MA12'] = df['OBV'].rolling(window=12).mean()
    df['OBV_diff'] = df['OBV'] - df['OBV_MA12']
    
    return df
 
   

In [6]:
df= VR_OBV(etf)

/Users/LarryGuo/miniconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/LarryGuo/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
df.columns

Index(['code', 'date', 'name', 'open', 'high', 'low', 'close', 'volume',
       'vol_up', 'vol_dn', 'VR', 'prev_close', 'diff', 'diff_index', 'OBV',
       'OBV_MA12', 'OBV_diff'],
      dtype='object')

In [8]:
df[24:]

,code,date,name,open,high,low,close,volume,vol_up,vol_dn,VR,prev_close,diff,diff_index,OBV,OBV_MA12,OBV_diff
24,50,20130205,元大台灣50,54.90,55.10,54.90,55.00,15988,206118.0,132218.0,0.430059,55.20,-0.20,-1,0.058682,0.026497,0.032185
25,50,20130206,元大台灣50,55.20,55.50,55.20,55.40,29434,248006.5,140147.5,0.513384,55.00,0.40,1,0.076855,0.032046,0.044809
26,50,20130218,元大台灣50,55.50,55.90,55.45,55.60,29739,276863.5,149766.5,0.544638,55.40,0.20,1,0.095216,0.038585,0.056631
27,50,20130219,元大台灣50,55.60,55.75,55.55,55.65,16817,298456.0,147276.0,0.614987,55.60,0.05,1,0.105599,0.046364,0.059235
28,50,20130220,元大台灣50,55.90,56.20,55.90,56.10,21814,325510.0,141182.0,0.725369,55.65,0.45,1,0.119067,0.055847,0.063220
29,50,20130221,元大台灣50,55.90,56.05,55.70,55.70,9951,326225.0,143327.0,0.713695,56.10,-0.40,-1,0.112923,0.065258,0.047666
30,50,20130222,元大台灣50,55.60,55.85,55.35,55.50,9130,317728.0,152668.0,0.636605,55.70,-0.20,-1,0.107286,0.073750,0.033536
31,50,20130223,元大台灣50,55.70,55.90,55.65,55.80,6375,294331.0,144869.0,0.617042,55.50,0.30,1,0.111222,0.081440,0.029782
32,50,20130225,元大台灣50,55.80,55.80,55.55,55.70,10431,267038.5,149679.5,0.519102,55.80,-0.10,-1,0.104782,0.087478,0.017304
33,50,20130226,元大台灣50,55.25,55.40,55.05,55.10,6753,251450.5,153487.5,0.461430,55.70,-0.60,-1,0.100613,0.092519,0.008094
